In [ ]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff pyyaml facebook_business pystache

In [1]:
%load_ext autoreload
%autoreload 2

In [54]:
from environs import Env

from adopt.malaria import get_confs, get_df, load_basics, window, get_db_conf
from adopt.marketing import make_audience_conf, dict_from_nested_type
from adopt.marketing import Marketing
from adopt.facebook.state import CampaignState, get_api
from adopt.malaria import load_typed_json
from adopt.marketing import AudienceConf, CampaignConf
from adopt.campaign_queries import get_campaigns_for_user, create_campaign_confs, create_campaign_for_user
from typing import List
import json

env = Env()
env.read_env('.env-vlab-vlab', override=True)
db_conf = get_db_conf(env)

In [55]:
USER = "mchatila@worldbank.org"
CAMPAIGN = "vaccination-iraq"
CAMPAIGNID = next(c['id'] for c in get_campaigns_for_user(USER, db_conf) 
                  if c['name'] == CAMPAIGN)

# create_campaign_for_user(USER, CAMPAIGN, db_conf)

In [76]:
userinfo, config, db_conf, state, m, confs = load_basics(CAMPAIGNID, env)

In [ ]:
state.campaign

In [82]:
template_state = CampaignState(userinfo.token, get_api(env, userinfo.token), '282370023153169', 'embed-mena-iraq-templates')

template_state.campaign_state

INFO:root:Loaded 24 adsets from campaign embed-mena-iraq-templates


INFO:root:Loaded 0 ads from campaign embed-mena-iraq-templates


[(<AdSet> {
      "bid_strategy": "LOWEST_COST_WITHOUT_CAP",
      "daily_budget": "2000",
      "destination_type": "MESSENGER",
      "id": "23847002574860281",
      "name": "Iraq- GEO - Saladin Governorate",
      "optimization_goal": "LINK_CLICKS",
      "optimization_sub_event": "NONE",
      "status": "ACTIVE",
      "targeting": {
          "age_max": 65,
          "age_min": 18,
          "brand_safety_content_filter_levels": [
              "FACEBOOK_STANDARD"
          ],
          "geo_locations": {
              "location_types": [
                  "home",
                  "recent"
              ],
              "regions": [
                  {
                      "country": "IQ",
                      "key": "1775",
                      "name": "Saladin Governorate"
                  }
              ]
          },
          "targeting_optimization": "none"
      }
  },
  []),
 (<AdSet> {
      "bid_strategy": "LOWEST_COST_WITHOUT_CAP",
      "daily_budget": "2000",
 

In [83]:
import pandas as pd

share_lookup = pd.read_csv('outs/iraq-distribution.csv', header=[0,1], index_col=[0])

In [85]:
from mena.strata import get_groups

groups = get_groups(template_state, share_lookup)

In [77]:
images = {i['name']: i for i in state.account.get_ad_images(fields=['name', 'hash'])}

In [ ]:
images

# CREATING CONFS - MNM

In [41]:
# make creatives conf
from adopt.marketing import CreativeConf


In [ ]:
# with tt as (with t as (select row_to_json(json_each(details)) d from (select details from adopt_reports where campaignid = '872cc8af-7b11-4a35-8ae0-b0a64059278b' order by created desc limit 1)) select d->>'key' as stratum, (d->'value'->>'goal')::float as goal,  (d->'value'->>'respondent_share')::float as share, (d->'value'->>'budget')::float as budget from t)  select *, goal - share as dif from tt order by dif desc;

# with tt as (with t as (select row_to_json(json_each(details)) d, created from (select details, adopt_reports.created from adopt_reports join campaigns on campaignid = campaigns.id where campaigns.name = 'vaccination-iraq')) select created, d->>'key' as stratum, (d->'value'->>'goal')::float as goal,  (d->'value'->>'respondent_share')::float as share, (d->'value'->>'budget')::float as budget from t) select created, sqrt(sum((goal - share)^2)) as dif from tt group by created order by created;

In [ ]:
c = {'optimization_goal': 'LINK_CLICKS',
     'destination_type': 'MESSENGER',
     'adset_hours': 48,
     'budget': 400000.0,
     'min_budget': 100.0,
     'opt_window': 2*24,
     'end_date': '2021-02-21',
     'proportional': True,
     'page_id': '101734981838685',
     'instagram_id': None,
     'ad_account': '282370023153169',
     'ad_campaign': 'embed-mena-iraq-1'}

config = CampaignConf(**c)

create_campaign_confs(CAMPAIGNID, "opt", [config._asdict()], db_conf)

In [49]:
RESPONDENT_AUDIENCE = "vlab-embed-iraq-vaccination-respondents"

In [ ]:
from adopt.marketing import make_audience_conf, dict_from_nested_type
import json

audiences = [
    {
        "name": RESPONDENT_AUDIENCE,
        "shortcodes": ["vaccwelcomeiraq"],
        "subtype": "CUSTOM"
    },
    {
        "name": "vlab-embed-iraq-low-education",
        "shortcodes": ['vaccinationiraqen', 'vaccinationiraqar', 'vaccinationkrd'],
        "subtype": "LOOKALIKE",
        "question_targeting": {
            "op": "not_equal",
            "vars": [
                {"type": "response", "value": "education"},
                {"type": "constant", "value": "D"}
            ]
        },
        "lookalike": {
            "name": "vlab-embed-mena-iraq-low-education-lookalike",
            "target": 2000,
            "spec": {
                "country": "IQ",
                "starting_ratio": 0.0,
                "ratio": 0.2
            }
        },
    },
    {
        "name": "vlab-embed-iraq-high-education",
        "shortcodes": ['vaccinationiraqen', 'vaccinationiraqar', 'vaccinationkrd'],
        "subtype": "LOOKALIKE",
        "question_targeting": {
            "op": "equal",
            "vars": [
                {"type": "response", "value": "education"},
                {"type": "constant", "value": "D"}
            ]
        },
        "lookalike": {
            "name": "vlab-embed-mena-iraq-high-education-lookalike",
            "target": 2000,
            "spec": {
                "country": "IQ",
                "starting_ratio": 0.0,
                "ratio": 0.2
            }
        },
    },
]


audience_confs = [make_audience_conf(c) for c in audiences]
confs = [dict_from_nested_type(a) for a in audience_confs]

create_campaign_confs(CAMPAIGNID, "audience", confs, db_conf)

In [ ]:
from adopt.marketing import CreativeConf


def _creative_conf(name, image, body, headline, welcome_message, button_text, form):
    return {
        "name": name,
        "image": image['name'],
        "image_hash": image['hash'],
        "body": body,
        "link_text": headline,
        "welcome_message": welcome_message,
        "button_text": button_text,
        "form": form,
    }

image_confs = [
    ('vlab-embed-iraq-back-to-normal-giveaway', 
     images['Giveaway_Op_2_Arabic.jpg'],
     'ساعدونا لاعادة الحياة الى طبيعتها! أجب عن بعض الأسئلة حول لقاح فيروس كورونا',
     'شارك رأيك',
     'شارك رأيك واحصل على فرصة للفوز بقسيمة شرائية بقيمة 150,000 دينار عراقي انقر ابدأ لتبدء',
     'ابدأ'
     ),
    ('vlab-embed-iraq-back-to-normal-opinion-1', 
     images['Share-your-opinion-1.png'],
     'ساعدونا لاعادة الحياة الى طبيعتها! أجب عن بعض الأسئلة حول لقاح فيروس كورونا',
     'شارك رأيك',
     'شارك رأيك واحصل على فرصة للفوز بقسيمة شرائية بقيمة 150,000 دينار عراقي انقر ابدأ لتبدء',
     'ابدأ'
     ),
    ('vlab-embed-iraq-back-to-normal-opinion-2', 
     images['Share-your-opinion-2.png'],
     'ساعدونا لاعادة الحياة الى طبيعتها! أجب عن بعض الأسئلة حول لقاح فيروس كورونا',
     'شارك رأيك',
     'شارك رأيك واحصل على فرصة للفوز بقسيمة شرائية بقيمة 150,000 دينار عراقي انقر ابدأ لتبدء',
     'ابدأ'
     ),
    ('vlab-embed-iraq-lottery', 
     images['0000_FB AD ARABIC.jpg'],
     '!شارك رأيك واحصل على فرصة لفوز قسيمة شرائية بقيمة 150000 الدينار العراقي',
     'شارك رأيك',
     'شارك رأيك واحصل على فرصة للفوز بقسيمة شرائية بقيمة 150,000 دينار عراقي انقر ابدأ لتبدء',
     'ابدأ'
     ),
]

creatives = [CreativeConf(**_creative_conf(*c, 'vaccwelcomeiraq')) for c in image_confs]
confs = [c._asdict() for c in creatives]

create_campaign_confs(CAMPAIGNID, "creative", confs, db_conf)

In [87]:
ALL_CREATIVES = [t[0] for t in image_confs]

def make_stratum(id_, quota, targeting):
    return { 'id': id_,
             'metadata': {'stratumid': id_, 'country': 'iraq'},
             'facebook_targeting': targeting, 
             'creatives': ALL_CREATIVES,
             'audiences': ['vlab-embed-mena-iraq-low-education-lookalike'],
             'excluded_audiences': [RESPONDENT_AUDIENCE],
             'quota': quota,
             'shortcodes': ['vaccinationiraqen', 'vaccinationiraqar', 'vaccinationkrd'],
             'question_targeting': {
                 'op': 'and',
                 'vars': [
                     { 'op': 'equal',
                       'vars': [{'type': 'response', 'value': 'md:stratumid'},
                                {'type': 'constant', 'value': id_}]},
                     { 'op': 'answered',
                       'vars': [{'type': 'response', 'value': 'age_cat'}]}
                 ]
             }}

In [ ]:
from adopt.marketing import make_stratum_conf

strata = [make_stratum(*g) for g in groups]
strata_data = [dict_from_nested_type(make_stratum_conf(c)) for c in strata]
create_campaign_confs(CAMPAIGNID, "stratum", strata_data, db_conf)

# TESTING

In [67]:
mal = load_basics(CAMPAIGNID, env)

In [71]:
mal.confs['audience']

[AudienceConf(name='vlab-embed-iraq-vaccination-respondents', subtype='CUSTOM', shortcodes=['vaccwelcomeiraq'], question_targeting=None, lookalike=None),
 AudienceConf(name='vlab-embed-iraq-low-education', subtype='LOOKALIKE', shortcodes=['vaccinationiraqen', 'vaccinationiraqar', 'vaccinationkrd'], question_targeting=QuestionTargeting(op='not_equal', vars=[TargetVar(type='response', value='education'), TargetVar(type='constant', value='D')], name=None), lookalike=Lookalike(name='vlab-embed-mena-iraq-low-education-lookalike', target=500, spec=LookalikeSpec(country='IQ', ratio=0.2, starting_ratio=0.0))),
 AudienceConf(name='vlab-embed-iraq-high-education', subtype='LOOKALIKE', shortcodes=['vaccinationiraqen', 'vaccinationiraqar', 'vaccinationkrd'], question_targeting=QuestionTargeting(op='equal', vars=[TargetVar(type='response', value='education'), TargetVar(type='constant', value='D')], name=None), lookalike=Lookalike(name='vlab-embed-mena-iraq-high-education-lookalike', target=500, spe

In [ ]:
%%time

from adopt.malaria import update_ads_for_campaign

instructions, report = update_ads_for_campaign(mal)

In [ ]:
from adopt.campaign_queries import create_adopt_report

db_conf['user'] = 'adopt'
create_adopt_report(CAMPAIGNID, 'FACEBOOK_ADOPT', report, db_conf)

In [8]:
import pandas as pd

rdf = pd.DataFrame(report).T

In [22]:
finals = (rdf.respondents + (rdf.budget / rdf.price)*4).reset_index(name = 'respondents')

In [26]:
finals['share'] = finals.respondents / finals.respondents.sum()

In [33]:
finals = finals.rename(columns={'index': 'stratum'}).sort_values('share', ascending=False)

In [ ]:
share_lookup[['1']].sum().sum()

In [ ]:
share_lookup[['2']].sum().sum()

In [ ]:
finals[finals.stratum.str.contains('-2-')].share.sum()

In [47]:
finals.to_csv('iraq-projection.csv', index=False)

In [ ]:
from adopt.malaria import run_instructions

run_instructions(instructions, mal.state)

In [5]:
from adopt.malaria import get_df

userinfo, config, db_conf, state, m, confs = mal
audience_confs = confs["audience"]

df = get_df(db_conf, userinfo.survey_user, audience_confs)

In [7]:
audience_confs

[AudienceConf(name='vlab-embed-iraq-vaccination-respondents', subtype='CUSTOM', shortcodes=['vaccwelcomeiraq'], question_targeting=None, lookalike=None)]

In [ ]:
df